In [1]:
from appium import webdriver
from appium.options.android import UiAutomator2Options
from appium.webdriver.common.appiumby import AppiumBy
from appium.webdriver.common.touch_action import TouchAction
from selenium.webdriver.common.action_chains import ActionChains, ActionBuilder
from selenium.webdriver.common.actions.pointer_input import PointerInput
from selenium.webdriver.common.actions import interaction

from bs4 import BeautifulSoup as bs
import re
import numpy as np
import pandas as pd
import time
from datetime import datetime

swipe_delay = 1
action_delay = 5
launch_delay = 3

# define swipe
def swipe(driver, start, end):
    actions = ActionChains(driver)
    actions.w3c_actions = ActionBuilder(driver, mouse=PointerInput(interaction.POINTER_TOUCH, "touch"))
    actions.w3c_actions.pointer_action.move_to_location(start[0], start[1])
    actions.w3c_actions.pointer_action.pointer_down()
    actions.w3c_actions.pointer_action.move_to_location(end[0], end[1])
    actions.w3c_actions.pointer_action.release()
    actions.perform()

def click(driver, x, y):
    actions = ActionChains(driver)
    actions.w3c_actions = ActionBuilder(driver, mouse=PointerInput(interaction.POINTER_TOUCH, "touch"))
    actions.w3c_actions.pointer_action.move_to_location(x, y)
    actions.w3c_actions.pointer_action.pointer_down()
    actions.w3c_actions.pointer_action.pause(0.1)
    actions.w3c_actions.pointer_action.release()
    actions.perform()

def process_bukalapak(rows, category, brand):
    return (
        pd.DataFrame(rows)
        .rename(columns={
            0:'SKU', 
            1:'price',
            2:'note',
        })
        .assign(
            price = lambda x: x.price.str.replace('[Rp\.]','', regex=True),
            category = category,
            brand = brand,
        )
    )

In [2]:
# dont forget to run "appium --allow-cors" in terminal

options = UiAutomator2Options()
options.automationName = 'UiAutomator2'
options.udid = 'emulator-5554'
options.platformName = 'Android'
options.platformVersion = '12'
options.deviceName = 'bwphone'

# setup the driver
driver = webdriver.Remote('http://127.0.0.1:4723', options=options)

In [3]:
# remove the warnings
import warnings
warnings.filterwarnings('once')

In [4]:
# keeps clicking back until reach home screen
while driver.current_activity != '.NexusLauncherActivity':
    driver.press_keycode(5)
    time.sleep(action_delay)

# open the bukalapak app then wait
driver.activate_app('com.bukalapak.mitra')
time.sleep(launch_delay)

# click back
driver.press_keycode(4)
time.sleep(action_delay)

# click the burger icon
click(driver, 1000, 150)
time.sleep(action_delay)
# swipe up
swipe(driver, (500, 1400), (500, 800))
time.sleep(swipe_delay)
# click "Harga Jual Produk Virtual"
[i for i in driver.find_elements(by=AppiumBy.CLASS_NAME, value='android.widget.TextView') if 'Harga Jual Produk Virtual' in i.text][0].click()
time.sleep(action_delay)


rows = []
categories = {'Pulsa':'Telkomsel', 'Paket Data':'Telkomsel', 'Top Up Digital':'OVO'}
for cat in categories:
    # click the product then wait
    [i for i in driver.find_elements(by=AppiumBy.CLASS_NAME, value='android.widget.TextView') if cat in i.text][0].click()
    time.sleep(action_delay)
    # close the first box first
    [i for i in driver.find_elements(by=AppiumBy.CLASS_NAME, value='android.widget.TextView') if categories[cat] in i.text][0].click()
    time.sleep(action_delay)
    # scrape
    brands = [i.text for i in driver.find_elements(by=AppiumBy.CLASS_NAME, value='android.widget.TextView')][3:]
    for brand in brands:
        [i for i in driver.find_elements(by=AppiumBy.CLASS_NAME, value='android.widget.TextView') if brand in i.text][0].click()
        time.sleep(action_delay)
        sku_list = []
        while True:
            source = driver.page_source
            soup = bs(source, 'html')
            new_rows = pd.concat([pd.Series(re.findall(r'text="(.+)"', str(i.parent))) for i in list(soup.find_all(attrs={'text': re.compile(r'^Rp.+$')}))], axis=1).T
            if set(new_rows[0]).intersection(sku_list) == set(new_rows[0]):
                break
            rows.append(process_bukalapak(new_rows, category=cat, brand=brand))
            sku_list.extend(new_rows[0])
            swipe(driver, (500, 1400), (500, 800))
            time.sleep(swipe_delay)
    # click back
    driver.press_keycode(4)
    time.sleep(action_delay)


# -- TOKEN LISTRIK
# click the product then wait
[i for i in driver.find_elements(by=AppiumBy.CLASS_NAME, value='android.widget.TextView') if 'Token Listrik' in i.text][0].click()
time.sleep(action_delay)
# scrape
while True:
    source = driver.page_source
    soup = bs(source, 'html')
    new_rows = pd.concat([pd.Series(re.findall(r'text="(.+)"', str(i.parent))) for i in list(soup.find_all(attrs={'text': re.compile(r'^Rp.+$')}))], axis=1).T
    if set(new_rows[0]).intersection(sku_list) == set(new_rows[0]):
        break
    rows.append(process_bukalapak(new_rows, category='Token Listrik', brand='PLN'))
    sku_list.extend(new_rows[0])
    swipe(driver, (500, 1400), (500, 800))
    time.sleep(swipe_delay)
# click back
driver.press_keycode(4)
time.sleep(action_delay)





c:\Users\bukuw\anaconda3\envs\workenv\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
c:\Users\bukuw\anaconda3\envs\workenv\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` i

In [5]:
results = (
    pd.concat(rows)
    .drop_duplicates()
    .dropna() # remove NaN
)

In [6]:
results

,SKU,price,note,category,brand
0,5.000,5340,Set harga,Pulsa,Telkomsel
1,10.000,10290,Set harga,Pulsa,Telkomsel
2,15.000,14950,Set harga,Pulsa,Telkomsel
3,20.000,20185,Set harga,Pulsa,Telkomsel
4,25.000,25025,Set harga,Pulsa,Telkomsel
...,...,...,...,...,...
1,50.000,50090,Set harga,Token Listrik,PLN
2,100.000,100090,Set harga,Token Listrik,PLN
3,200.000,200090,Set harga,Token Listrik,PLN
4,500.000,500090,Set harga,Token Listrik,PLN


In [7]:
### OLD CODE - REPLACED IN SEPT 14, 2023
# # keeps clicking back until reach home screen
# while driver.current_activity != '.NexusLauncherActivity':
#     driver.press_keycode(4)
#     time.sleep(action_delay)

# # open the bukalapak app then wait
# driver.activate_app('com.bukalapak.mitra')
# time.sleep(launch_delay)

# # dismiss login
# driver.find_element(by=AppiumBy.XPATH, value='//android.widget.TextView[@text="Gabung sekarang!"]').click()
# time.sleep(action_delay)
# driver.find_element(by=AppiumBy.XPATH, value='//android.widget.TextView[@text="Lewati"]').click()
# time.sleep(action_delay)

# # press pulsa -> access to pulsa, paket, token listrik
# driver.find_element(by=AppiumBy.XPATH, value='//android.widget.TextView[@text="Pulsa"]').click()
# time.sleep(action_delay)

# results = []
# for category in ['Pulsa', 'Paket Data']:
#     driver.find_element(by=AppiumBy.XPATH, value=f'//android.widget.TextView[@text="{category}"]').click()    

#     brands = [i.text for i in driver.find_elements(by=AppiumBy.XPATH, value='//android.widget.LinearLayout/android.widget.TextView')]
#     for brand in brands:
#         print(f'scrap {brand}')
#         driver.find_element(by=AppiumBy.XPATH, value=f'//android.widget.LinearLayout/android.widget.TextView[@text="{brand}"]').click()

#         rows = []
#         while True:
#             source = driver.page_source
#             soup = bs(source, 'html.parser')
#             new_rows = [tuple(re.findall(r'text="(.+)"', str(i.parent))) for i in list(soup.find_all(attrs={'text': re.compile(r'^Rp.+$')}))]
#             if set(new_rows).intersection(rows) == set(new_rows):
#                 break
#             swipe(driver, (500, 1400), (500, 800))
#             time.sleep(swipe_down_delay)
#             rows.extend(new_rows)
#         results.append(process_bukalapak(rows, category, brand))

#         while True:
#             try:
#                 el = driver.find_element(by=AppiumBy.XPATH, value=f'//android.widget.LinearLayout/android.widget.TextView[@text="{brand}"]')
#                 time.sleep(action_delay)
#                 el.click()
#                 break
#             except:
#                 swipe(driver, (500, 800), (500, 1400))
#                 time.sleep(swipe_up_delay)

# driver.find_element(by=AppiumBy.XPATH, value=f'//android.widget.TextView[@text="Token Listrik"]').click()
# time.sleep(action_delay)
# rows = []
# while True:
#     source = driver.page_source
#     soup = bs(source, 'html.parser')
#     new_rows = [tuple(re.findall(r'text="(.+)"', str(i.parent))) for i in list(soup.find_all(attrs={'text': re.compile(r'^Rp.+$')}))]
#     if set(new_rows).intersection(rows) == set(new_rows):
#         break
#     swipe(driver, (500, 1400), (500, 800))
#     time.sleep(swipe_down_delay)
#     rows.extend(new_rows)
# results.append(process_bukalapak(rows, category='Token Listrik', brand=None))

# results_df = (
#     pd.concat(results)
#     .drop_duplicates()
#     .assign(
#         platform = 'mitra bukalapak',
#     )
# )

In [9]:
from datetime import date
today = date.today().strftime("%Y_%m_%d")
results.to_csv(f'result/bukalapak_scrape_{today}.csv')